# Black Scholes Model

The Black-Scholes Model was developed by professor Fisher Black, Myron Scholes, and Robert Merton. The model is used to value option contracts. The model makes the following assumptions: 
- Lognormal distribution for the stock price
- There are no transaction costs or taxes
- There is no cash flow from the underlying asset during the time to maturity of the option
- There are no arbitrage opportunities
- Investors can borrow or lend at the risk-free rate 
- The risk-free rate is constant
- The model is based on European-style options


The key concept from the mathematical model is that the option and the stock price depend on the same underlying source of uncertainty. This uncertainty could be eliminated by constructing a portfolio with the stock and the option. This portfolio should be instantaneously riskless and earn the risk-free rate. The portfolio will require continuous rebalancing.

The mathematical formula to calculate the call price of a European option is:

\begin{equation*}
C = S_t N \left(d1 \right) - K e^{-rt} N \left(d2 \right)
\end{equation*}

__Where:__

\begin{equation*}
d1 = \frac{ \ln{ \frac{S_t}{K} + \left(r + \frac{\sigma_u^2}{2} \right) t} }{\sigma_s \sqrt{t}}
\end{equation*}

and:

\begin{equation*}
d2 = d1 -  \sigma_s \sqrt{t}
\end{equation*}


__Where:__

\begin{alignat*}{1}
&C \quad & = \quad & \text{Call option price} \\
&S \quad & = \quad & \text{Current stock (or other underlying) price} \\
&K \quad & = \quad & \text{Strike price} \\
&r \quad & = \quad & \text{Risk-free interest rate} \\
&t \quad & = \quad & \text{time to maturity} \\
&N \quad & = \quad & \text{A normal distribution}
\end{alignat*}

In this notebook, I will develop a process to calculate the call price of any option given all the parameters required in the Black-Scholes model. To do that, I will use Python, specifically Yfinance and Quandl, libraries to obtain the required data.

First, let's import the necessary libraries to make the analysis.

__Note:__ Some libraries might not be pre-installed so the reader will need to uncomment the line code to install them. 

In [1]:
#pip install stockquotes
#pip install Quandl

import math
import scipy.stats as st
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import stockquotes
import quandl
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from BlackScholesMerton import BlackScholesModel
import os
from scipy.optimize import minimize

%matplotlib inline

I will use the Quandl library to obtain information about the risk-free rate. To use Quandl, you will need to set up an account and get an API key. The code to call Quandl API is <code>quandl.ApiConfig.api_key = 'YOUR KEY GOES HERE'</code>

In [2]:
quandl.ApiConfig.api_key = os.environ.get('QUANDL_KEY')

From the YFinance library, I will get the historical information of the S&P500.

This library also allows getting the expiration dates for the options.

Based on the expirations, I can get the option chain with the information required to calculate the option price using the Black-Scholes model. Let's have a look at the information available for the option chain on the S&P 500 with expiration on December 17, 2020.

This analysis will focus only on calculating the call price; however, once a put price formula is defined the reader can easily run the code with all the variables already defined in this notebook. Let's have a look at the DataFrame generated from YFinance.

From this dataFrame, I need the information related to the strike price and the implied volatility.

We can save the strike price in a dictionary where the keys will be the different expiration dates and the values will be an array with all the strike prices available on the specific expiration date. 

Similarly, I can get a dictionary with information related to the implied volatility. Once again, the keys of the dictionary will be the different expiration dates and the values of the dictionary will be an array with the implied volatility corresponding to a determined strike price.

In [ ]:
# sigma = {name: pd.DataFrame() for name in expirations}

# for i, a in enumerate(expirations):
#     GSPC_opts = GSPC.option_chain(expirations[i])
#     GSPC_calls = GSPC_opts.calls
#     sigma[a] = GSPC_calls['impliedVolatility']

With the same logic used before, I can get the time to maturity for each expiration date. I will define a dictionary where the keys are the different expiration dates available, and the values of the dictionary are the difference between the current date and the expiration date. However, in this case, values are not an array of data, instead, the values are a scalar.

In [ ]:
# t = {name: pd.DataFrame() for name in expirations}

# for i, a in enumerate(expirations):
#     expiration_date = datetime.strptime(expirations[i],"%Y-%m-%d") 
#     today = datetime.now()
#     t[a] = (expiration_date - today).days/365

Using the Quandl library, I can get access to the daily Treasury Yield Curve and saved it in a variable called <code>yield_curve</code>. To get the yield curve, I used the get module as follows: <code>quandl.get("USTREASURY/YIELD", authtoken="YOUR TOKEN GOES HERE")</code>

In [ ]:
# yield_curve = quandl.get("USTREASURY/YIELD", authtoken = os.environ.get('QUANDL_KEY'))
# yield_curve

Now, I will define a function that allows me to calculate the time to maturity of the option based on the expiration date. The next step will be to match this time to maturity with the most appropriate treasury yield.

To achieve this matching task, I will define a function that browses through the list of available treasury maturities and matches the time to maturity of the option with the corresponding yield of the closest greater treasury maturity.

In [ ]:
# def TTM(exp_date,today):
#     time_to_maturity = (exp_date - today).days
#     return time_to_maturity

# def rf(exp_date,today):
#     num_days = [30,60,90,180,365,730,1095,1825,2555,3650,7300,10950]
#     rf_ttm = list(yield_curve.columns)
#     risk_free_L = []
#     if round(TTM(exp_date,today)) < 0:
#         return "Expire date must be greater than today"
#     elif round(TTM(exp_date,today)) > num_days[-1]: 
#         return yield_curve[rf_ttm[-1]][-1]/100
#     else:
#         for b, f in zip(num_days, rf_ttm):
#                 if round(TTM(expiration_date,today)) < b:
#                     risk_free_L.append(yield_curve[f][-1]/100)
#         return risk_free_L[0]

With this last function defined, I can create a dictionary with the different risk-free rates relevant for each expiration. Again, the keys of the dictionary will be the expirations of the option and the values will be a scalar with the corresponding risk-free rate.

In [ ]:
# rf_rate = {name: pd.DataFrame() for name in expirations}

# for i, a in enumerate(expirations):
#     expiration_date = datetime.strptime(expirations[i],"%Y-%m-%d") 
#     today = datetime.now()
#     rf_rate[a] = rf(expiration_date,today)

The last variable needed to calculate the price of the call option is the current price of the security. This can be obtained from the stockquotes library through the <code>Stock</code> module.

Now, let's develop the Black-Scholes model. I will define the model with 3 different functions; the advantage of this is that these functions could potentially  be used to create a Black-Scholes Class in Python for future development projects.

The first function will calculate the expected value of the stock if the option is exercised using risk-adjusted probabilities. This is the equivalent of the term $\begin{equation*}N \left(d1 \right)\end{equation*}$. The function will return a tuple containing $\begin{equation*}d1 \end{equation*}$ and $\begin{equation*}N \left(d1 \right)\end{equation*}$.

In [ ]:
# def Norm_d1(S, K, r, t, sigma):
#     d1 = (np.log(S/K)+(r+((np.power(sigma,2)/2))*t))/(sigma*math.sqrt(t))
#     N_d1 = st.norm.cdf(d1)
#     return (d1, N_d1)

The next function that I will define is the probability of the function to be exercised based on risk-adjusted probabilities. This is equivalent to the term $\begin{equation*}N \left(d2 \right)\end{equation*}$.

In [ ]:
# def Norm_d2(d1, sigma, t):
#     d2 = d1 - sigma*math.sqrt(t)
#     N_d2 = st.norm.cdf(d2)
#     return N_d2

Lastly, I will define the function that will calculate the call price taking an input all the variables required in the model and using the previous two functions.

In [ ]:
# def Call_price(S, K, r, t, sigma):
#     d1, N_d1 = Norm_d1(S, K, r, t, sigma)
#     N_d2 = Norm_d2(d1, sigma, t)
#     C = S*N_d1-K*math.exp(-r*t)*N_d2
#     return C

With the call price function define, I can create a dictionary that saves the array of call prices for different expiration dates. 

Let's visualize the results. The following graph will compare, for the two expirations available, the call price for different strike prices calculated using the Black-Scholes model and the actual price taken from the YFinance library. 

The difference between the Black-Scholes model and the actual price could be explained by lack of liquidity, especially in options with low strike price, and/or differences in the implied volatility. 

__References:__<p>&nbsp;</p> 
- Investopedia (2020). Black Scholes Model https://www.investopedia.com/terms/b/blackscholes.asp
- John C. Hull (2016). Fundamentals of Futures and Options Markets, 9th Ed, Ch 13

# Implied Volatility

In [3]:
GSPC = yf.Ticker('^GSPC')

In [4]:
expirations = GSPC.options
expirations

('2020-12-17', '2021-12-16')

In [5]:
GSPC_opts = GSPC.option_chain(expirations[0])
GSPC_opts

Options(calls=        contractSymbol       lastTradeDate  strike  lastPrice  bid  ask  \
0   SPX201218C00100000 2020-08-10 19:17:13   100.0    3238.72  0.0  0.0   
1   SPX201218C00200000 2020-08-10 13:32:00   200.0    3136.14  0.0  0.0   
2   SPX201218C00300000 2020-07-09 20:55:27   300.0    2738.00  0.0  0.0   
3   SPX201218C00400000 2020-07-09 20:55:30   400.0    2638.40  0.0  0.0   
4   SPX201218C00500000 2020-07-27 12:00:10   500.0    2707.70  0.0  0.0   
..                 ...                 ...     ...        ...  ...  ...   
89  SPX201218C03800000 2020-08-11 19:56:23  3800.0      16.94  0.0  0.0   
90  SPX201218C03900000 2020-08-11 19:51:01  3900.0       9.57  0.0  0.0   
91  SPX201218C04000000 2020-08-11 19:58:23  4000.0       5.30  0.0  0.0   
92  SPX201218C04100000 2020-08-11 19:25:40  4100.0       3.20  0.0  0.0   
93  SPX201218C04200000 2020-08-11 20:12:53  4200.0       1.80  0.0  0.0   

      change  percentChange  volume  openInterest  impliedVolatility  \
0   0.000000 

In [6]:
GSPC_calls = GSPC_opts.calls
GSPC_calls

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
0,SPX201218C00100000,2020-08-10 19:17:13,100.0,3238.72,0.0,0.0,0.000000,0.000000,11,3270,0.000010,True,REGULAR,USD
1,SPX201218C00200000,2020-08-10 13:32:00,200.0,3136.14,0.0,0.0,0.000000,0.000000,9,3144,0.000010,True,REGULAR,USD
2,SPX201218C00300000,2020-07-09 20:55:27,300.0,2738.00,0.0,0.0,0.000000,0.000000,8,86,0.000010,True,REGULAR,USD
3,SPX201218C00400000,2020-07-09 20:55:30,400.0,2638.40,0.0,0.0,0.000000,0.000000,6,16,0.000010,True,REGULAR,USD
4,SPX201218C00500000,2020-07-27 12:00:10,500.0,2707.70,0.0,0.0,0.000000,0.000000,1,58,0.000010,True,REGULAR,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,SPX201218C03800000,2020-08-11 19:56:23,3800.0,16.94,0.0,0.0,-1.859999,-9.893610,108,13861,0.031260,False,REGULAR,USD
90,SPX201218C03900000,2020-08-11 19:51:01,3900.0,9.57,0.0,0.0,-1.430000,-13.000003,84,7540,0.062509,False,REGULAR,USD
91,SPX201218C04000000,2020-08-11 19:58:23,4000.0,5.30,0.0,0.0,-1.000000,-15.873014,197,11476,0.062509,False,REGULAR,USD
92,SPX201218C04100000,2020-08-11 19:25:40,4100.0,3.20,0.0,0.0,-0.380000,-10.614522,3,2117,0.062509,False,REGULAR,USD


In [7]:
strike = {name:name for name in expirations}

for i, a in enumerate(expirations):
    GSPC_opts = GSPC.option_chain(expirations[i])
    GSPC_calls = GSPC_opts.calls
    strike[a] = GSPC_calls['strike']
    

In [8]:
today = datetime.now()

In [9]:
SP500 = stockquotes.Stock("^GSPC")
stock_price = SP500.current_price

In [10]:
call_premium = {name:name for name in expirations}

for i, a in enumerate(expirations):
    GSPC_opts = GSPC.option_chain(expirations[i])
    GSPC_calls = GSPC_opts.calls
    call_premium[a] = GSPC_calls['lastPrice']

In [11]:
BSM = {name:name for name in expirations}

for i, a in enumerate(expirations):
    BSM[a] = BlackScholesModel(datetime.strptime(expirations[i],"%Y-%m-%d"),today,stock_price,strike[a], \
                               premium=call_premium[a])

In [12]:
maturity = {name:name for name in expirations}

for i, a in enumerate(expirations):
    maturity[a] = BSM[a].maturity()

In [13]:
maturity

{'2020-12-17': 0.34794520547945207, '2021-12-16': 1.3452054794520547}

In [14]:
difference = {name:name for name in expirations}

sigma = 0.4

for i, a in enumerate(expirations):
    difference[a] = BSM[a].difference(sigma)

In [15]:
difference['2020-12-17'][4]

15926.117824549317

In [ ]:
# implied_vol = {name: pd.DataFrame() for name in expirations}

# x0 = [1]

# for i, a in enumerate(expirations):
#     implied_vol[a] = minimize(BSM[a].difference(1), x0)

In [16]:
BSM = {name:name for name in expirations}

for i, a in enumerate(expirations):
    strikeP = {count:count for count in range(0,len(expirations))}
    for j in range(0,len(strike[a])):
        strikeP[j] = BlackScholesModel(datetime.strptime(expirations[i],"%Y-%m-%d"),today,stock_price,strike[a][j], \
                                   premium=call_premium[a][j])
    BSM[a] = strikeP

In [17]:
BSM

{'2020-12-17': {0: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74788>,
  1: <BlackScholesMerton.BlackScholesModel at 0x1fb77d744c8>,
  2: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74b48>,
  3: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74b08>,
  4: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74808>,
  5: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74e08>,
  6: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74088>,
  7: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74a08>,
  8: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74e88>,
  9: <BlackScholesMerton.BlackScholesModel at 0x1fb77d743c8>,
  10: <BlackScholesMerton.BlackScholesModel at 0x1fb77d746c8>,
  11: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74f08>,
  12: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74e48>,
  13: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74c08>,
  14: <BlackScholesMerton.BlackScholesModel at 0x1fb77d74c48>,
  15: <BlackScholesMerton.BlackScholesModel at 0x1f

In [20]:
import timeit

print(timeit.timeit('''BSM.difference''',number=1)) 

NameError: name 'BSM' is not defined

In [ ]:
implied_vol = {name:name for name in expirations}

x0 = 1

for i, a in enumerate(expirations):
    strikeP = {count:count for count in range(0,len(expirations))}
    for j in range(0,1):
        strikeP[j] = minimize(BSM[a][j].difference,x0)['x']
    implied_vol[a] = strikeP

In [ ]:
implied_vol

In [ ]:
for i, a in enumerate(expirations):
    for j in range(0,len(BSM[a])):
        x0 = 1
        minimize(BSM[a].difference,x0)

In [ ]:
bsm = BlackScholesModel(datetime.strptime(expirations[1],"%Y-%m-%d"),today,stock_price,strike['2021-12-16'][3],\
                        premium=call_premium['2021-12-16'][3])

In [ ]:
x0 = 1
minimize(bsm.difference,x0)

In [ ]:
price = 3271.12
strike = 1000
Risk_free = 0.0011
time_to_maturity = 1.356164383561644
Call_premium = 3006.29

In [ ]:
def Norm_d1(S, K, r, t, sigma):
    d1 = (np.log(S/K)+(r+((np.power(sigma,2)/2))*t))/(sigma*math.sqrt(t))
    N_d1 = st.norm.cdf(d1)
    return (d1, N_d1)

def Norm_d2(d1, sigma, t):
    d2 = d1 - sigma*math.sqrt(t)
    N_d2 = st.norm.cdf(d2)
    return N_d2

def Call_price(S, K, r, t, sigma):
    d1, N_d1 = Norm_d1(S, K, r, t, sigma)
    N_d2 = Norm_d2(d1, sigma, t)
    C = S*N_d1-K*math.exp(-r*t)*N_d2
    return C

def difference(sigma):
    d1, N_d1 = Norm_d1(price, strike, Risk_free, time_to_maturity, sigma)
    N_d2 = Norm_d2(d1, sigma, time_to_maturity)
    C = price*N_d1-strike*math.exp(-Risk_free*time_to_maturity)*N_d2
    return (Call_premium - C)**2

In [ ]:
x0 = [2]
minimize(difference, x0)